In [2]:
import pandas as pd
import sqlite3
import src.data_enrichment as enrich
import src.helpers as helpers
import src.login_config as login_config

In [2]:
# Read in data
conn = sqlite3.connect("housing_information.db")
data_df = pd.read_sql_query('SELECT * FROM df_listings', conn)

## Data enrichment

In [3]:
data_df['house_age'] = data_df['approx_age'].apply(enrich.age_of_house)
data_df['difference_in_days'] = data_df.apply(lambda row: enrich.calculate_days_to_sell(row['list_date'], row['end_date']), axis=1)
data_df['price_difference_abs'] = data_df.apply(lambda row: enrich.calculate_price_diff_or_pct(row['list_price'], row['sold_price'], 'difference'), axis=1)
data_df['price_difference_pct'] = data_df.apply(lambda row: enrich.calculate_price_diff_or_pct(row['list_price'], row['sold_price'], 'percentage'), axis=1)

## Data Exploration

### Address 

Currently the address name is retrieved in the following format:
<br> - "Property Address: {name of address}, {city}, {province}"
<br> - Example: 'Property Address: Juneau Street, Burnaby, British Columbia'
<br>
<br> We only need the {name of address}. We will perform regex in order to retrieve it.

In [4]:
# Clean up "property_addr" field
data_df['Address_Clean'] = data_df['property_addr'].apply(enrich.regex_cleanup)

The following addresses need to be updated. For some odd reason, the data scrape algorithm did not catch the full address of these names. We will need to manually check and verify the correct name of the address.

In [5]:
# few rows that needs to be replaced
# Juneau Street -> 1301 - 4488 Juneau Street
# Soball Street -> 38 - 1295 Soball Street
# Seaham Crescent -> 11300 Seaham Crescent
# Yale Street -> 2558 Yale Street [ But remove this since lot was bought]

data_df.loc[data_df['Address_Clean'] == 'Juneau Street','Address_Clean'] ='1301 - 4488 Juneau Street'
data_df.loc[data_df['Address_Clean'] == 'Soball Street','Address_Clean'] ='38 - 1295 Soball Street'
data_df.loc[data_df['Address_Clean'] == 'Seaham Crescent','Address_Clean'] ='11300 Seaham Crescent'
data_df = data_df[data_df.Address_Clean != 'Yale Street']

### Home type

In [6]:
data_df.house_type.value_counts()

house_type
Apartment/Condominium      7383
Single Family Residence    4341
Townhouse                  2758
Multi Family                345
Manufactured Home            43
Manufactured On Land         20
Duplex                       17
Quadruplex                    4
Triplex                       3
Condo Apt                     1
House                         1
Name: count, dtype: int64

Notice above that house type are categorized into multiple categories. However, the ones with low value counts actually below in one of the following categories:
<br> - Apartment/Condominium
<br> - Single Family Residence
<br> - Townhouse

In [7]:
data_df['Home_type'] = data_df['house_type'] 

data_df.loc[data_df['house_type'] == 'Quadruplex','Home_type'] ='House'
data_df.loc[data_df['house_type'] == 'Manufactured On Land','Home_type'] ='House'
data_df.loc[data_df['house_type'] == 'Condo Apt','Home_type'] ='Apartment/Condominium'
data_df.loc[data_df['house_type'] == 'Triplex','Home_type'] ='House'
data_df.loc[data_df['house_type'] == 'Duplex','Home_type'] ='House'
data_df.loc[data_df['house_type'] == 'Manufactured Home','Home_type'] ='House'
data_df.loc[data_df['house_type'] == 'Multi Family','Home_type'] ='House'
data_df.loc[data_df['house_type'] == 'Single Family Residence','Home_type'] ='House'

Let's verify that the changes have indeed been made

In [8]:
data_df.Home_type.value_counts()

Home_type
Apartment/Condominium    7384
House                    4774
Townhouse                2758
Name: count, dtype: int64

### Maintanience fee
Remove it as they are all blanks. We will revisit this later
<br> For Apartment/Condominum, we could fetch this data off of condos.ca. There may be a better way to retrieve maintenacne fee. In the meantime, let's not include this column in the master copy.

In [9]:
data_df.maint_fee.value_counts()

maint_fee
$843    1
Name: count, dtype: int64

### Levels

In [10]:
data_df.levels.value_counts()

levels
Residential           14891
Residential Income       24
Land                      3
Residential Lease         1
Apartment                 1
Detached                  1
Name: count, dtype: int64

Based on the distribution of the field "levels". It seems that this variable is not that useful as majority of the listings are classified as "Residential". Therefore we will not include this variable in the final dataframe to work with.

### Garage 

In [11]:
data_df.garage.value_counts()

garage
Yes          12830
No            2089
Undergrnd        1
Name: count, dtype: int64

Notice there is one row that lists as "Undergrnd". Will update from "Undergrnd" to "Yes".

In [12]:
data_df.loc[data_df['garage'] == 'Undergrnd','garage'] ='Yes'

Verification

In [13]:
data_df.garage.value_counts()

garage
Yes    12831
No      2089
Name: count, dtype: int64

### House age
There are 6 rows (listings) where house age is not calculated. This it because no built age was provided.

In [14]:
print('The total # of rows before dropping nas is:', len(data_df)) # Before dropping nas
data_df = data_df.dropna(axis = 0, subset = ['house_age']).reset_index(drop = True)
print('The total # of rows after dropping nas is:', len(data_df)) # After dropping nas --  Before dropping nas - After dropping nas = 6 rows

The total # of rows before dropping nas is: 14921
The total # of rows after dropping nas is: 14915


Create master dataframe

In [15]:
columns_to_keep = ['mls_number', 'Address_Clean', 'City', 'Home_type', 'approx_age', 'house_age', 
                   'garage','garage_size','taxes', 'avg_price_sqft', 'bedroom','bathroom','list_date',
                   'list_price','end_date','sold_price', 'difference_in_days','price_difference_abs', 
                   'price_difference_pct']

df_master = data_df[columns_to_keep].copy()

df_master.rename(columns = {'Address_Clean': 'address',
                            'approx_age': 'yr_built',
                            'house_age': 'home_age'}, inplace = True)
df_master.columns = map(str.lower, df_master.columns)

## Features

In [30]:
import googlemaps
gmaps = googlemaps.Client(key = login_config.GEOCODE_KEY)

In [68]:
df_master['geocode_json'] = df_master.apply(lambda row: enrich.geocode_json(row['address'], row['city'], gmaps), axis=1)
df_master['geocode_json'] = df_master['geocode_json'].astype(str)

In [69]:
# #Store this in a database, # saving progress

# # conn = sqlite3.connect("housing_information.db")

# ##push the dataframe to sql 
# df_master.to_sql("df_master", conn, if_exists="replace", index = False)

# ##create the table

# conn.execute(
#     """
#     create table my_table_master as 
#     select * from df_master
#     """)

In [8]:
# # Read data back in
# # Read in data
# conn = sqlite3.connect("housing_information.db")
# df_master = pd.read_sql_query('SELECT * FROM df_master', conn)

In [9]:
# import ast
df_master['geocode_json'] = df_master['geocode_json'].apply(ast.literal_eval)
df_master['postal_code'] = df_master['geocode_json'].apply(enrich.extract_geocode_postal)

In [ ]:
df_master['index_col'] = range(0,len(df_master))

There are some rows where the postal code is not length 7. By definition, in Canada, postal code should be in the following format: 
<br> - A1A 1A1, where A is letter and 1 is a digit, with a space separating the third and fourth characters. Therefore the length of the postal code is 7.

In [32]:
filtered_df_equal_7  = df_master[df_master['postal_code'].str.len() == 7]
print(f"The number of rows where postal code == 7 is: {len(filtered_df_equal_7 )}")

filtered_df_not_equal_7 = df_master[df_master['postal_code'].str.len() != 7]
print(f"The number of rows where postal code != 7 is: {len(filtered_df_not_equal_7 )}")

The number of rows where postal code == 7 is: 13838
The number of rows where postal code != 7 is: 1077


There are a total of 1077 rows (listings) where the postal codes do not match the format of a canadian postal code. Let's explore.

Example #1:

In [36]:
filtered_df_not_equal_7['geocode_json'][38] # Doesn't seem like searching address with just street + city is enough.

[{'address_components': [{'long_name': 'Burnaby',
    'short_name': 'Burnaby',
    'types': ['locality', 'political']},
   {'long_name': 'Metro Vancouver',
    'short_name': 'Metro Vancouver',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'British Columbia',
    'short_name': 'BC',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'Canada',
    'short_name': 'CA',
    'types': ['country', 'political']}],
  'formatted_address': 'Burnaby, BC, Canada',
  'geometry': {'bounds': {'northeast': {'lat': 49.2993349, 'lng': -122.891689},
    'southwest': {'lat': 49.180637, 'lng': -123.0246499}},
   'location': {'lat': 49.2488091, 'lng': -122.9805104},
   'location_type': 'APPROXIMATE',
   'viewport': {'northeast': {'lat': 49.2993349, 'lng': -122.891689},
    'southwest': {'lat': 49.180637, 'lng': -123.0246499}}},
  'partial_match': True,
  'place_id': 'ChIJc4OcIah3hlQRabFSh3NyCt0',
  'types': ['locality', 'political']}]

In [47]:
# Example 2:
filtered_df_not_equal_7['geocode_json'][14911] # seems like there is a similar location in the USA. There is a city in USA called Vancouver

[{'address_components': [{'long_name': '365',
    'short_name': '365',
    'types': ['street_number']},
   {'long_name': 'East 16th Street',
    'short_name': 'E 16th St',
    'types': ['route']},
   {'long_name': 'Esther Short',
    'short_name': 'Esther Short',
    'types': ['neighborhood', 'political']},
   {'long_name': 'Vancouver',
    'short_name': 'Vancouver',
    'types': ['locality', 'political']},
   {'long_name': 'Clark County',
    'short_name': 'Clark County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Washington',
    'short_name': 'WA',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '98663', 'short_name': '98663', 'types': ['postal_code']},
   {'long_name': '3410',
    'short_name': '3410',
    'types': ['postal_code_suffix']}],
  'formatted_address': '365 E 16th St, Vancouver, WA 98663, USA',
  'geometry': {'l

Let's re-run the addresses in filtered_df_not_equal_7, where postal codes did not have length == 7. Let's tweak our function enrich.geocode_json to include provincial and country during the search of the geocode json

In [53]:
filtered_df_not_equal_7['postal_code_correction'] = filtered_df_not_equal_7.apply(lambda row: enrich.geocode_json(row['address'], row['city'], gmaps), axis=1)
filtered_df_not_equal_7['postal_code_correction'] = filtered_df_not_equal_7['geocode_json_rerun'].apply(enrich.extract_geocode_postal)

C:\Users\Alan\AppData\Local\Temp\ipykernel_16588\211698072.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_not_equal_7['postal_code_correction'] = filtered_df_not_equal_7.apply(lambda row: enrich.geocode_json(row['address'], row['city'], gmaps), axis=1)


In [88]:
# rerun_filtered_df_equal_7  = filtered_df_not_equal_7[filtered_df_not_equal_7['postal_code_correction'].str.len() == 7]
# print(f"The number of rows where postal code == 7 is: {len(rerun_filtered_df_equal_7 )}")

# rerun_filtered_df_not_equal_7 = filtered_df_not_equal_7[filtered_df_not_equal_7['postal_code_correction'].str.len() != 7]
# print(f"The number of rows where postal code != 7 is: {len(rerun_filtered_df_not_equal_7 )}")

In [89]:
# rerun_filtered_df_not_equal_3 = rerun_filtered_df_not_equal_7[rerun_filtered_df_not_equal_7['postal_code_correction'].str.len() != 3]
# print(f"The number of rows where postal code != 3 is: {len(rerun_filtered_df_not_equal_3 )}")

In [112]:
df1 = filtered_df_equal_7
cols_to_add = ['mls_number', 'address', 'city', 'home_type', 'yr_built', 'home_age',
       'garage', 'garage_size', 'taxes', 'avg_price_sqft', 'bedroom',
       'bathroom', 'list_date', 'list_price', 'end_date', 'sold_price',
       'difference_in_days', 'price_difference_abs', 'price_difference_pct',
       'geocode_json_rerun', 'postal_code_correction', 'index_col']
df2 = filtered_df_not_equal_7[cols_to_add]
df2 = df2.rename(columns = {'geocode_json_rerun': 'geocode_json',
                  'postal_code_correction': 'postal_code'})
# geocode_json_rerun, postal_code_correction
remaster_df = pd.concat([df1, df2])
remaster_df = remaster_df.sort_values(by = ['index_col'], ascending = True)

In [116]:
remaster_df['geocode_json'] = remaster_df['geocode_json'].astype(str)

In [31]:
# #Store this in a database, # saving progress

# conn = sqlite3.connect("housing_info.db")

# ##push the dataframe to sql 
# df_master.to_sql("df_master", conn, if_exists="replace", index = False)

# ##create the table

# conn.execute(
#     """
#     create table my_table as 
#     select * from df_master
#     """)

In [23]:
import ast
conn = sqlite3.connect("housing_info.db")
df_master = pd.read_sql_query('SELECT * FROM df_master', conn)
df_master['geocode_json'] = df_master['geocode_json'].apply(ast.literal_eval)

In [4]:
# conn = sqlite3.connect("housing_dataframes.db")
# cursor = conn.cursor()
# cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(cursor.fetchall())

In [5]:
# cursor.execute("DROP TABLE my_table_master;")

In [24]:
filtered_df_equal_7  = df_master[df_master['postal_code'].str.len() == 7]
print(f"The number of rows where postal code == 7 is: {len(filtered_df_equal_7 )}")

filtered_df_not_equal_7 = df_master[df_master['postal_code'].str.len() != 7]
print(f"The number of rows where postal code != 7 is: {len(filtered_df_not_equal_7 )}")

The number of rows where postal code == 7 is: 14397
The number of rows where postal code != 7 is: 518


Of the postal codes where the length is not equal to 7 (Canadian format). Let's determine what the formats are like

In [25]:
filtered_df_not_equal_7.postal_code

38       None
62        V5A
84        V5C
153       V5G
162       V5H
         ... 
14871    None
14893    None
14900    None
14907    None
14913     V5T
Name: postal_code, Length: 518, dtype: object

### Postal Code format description
We can see that some postal codes have the first three characters of the postal code. According to the definition of a canadian postal code, the first three characters of a postal determine the following: 
<br> - First letter denotes a particular "postal district", which outside of Quebec and Ontario, corresponds to an entire province or territory.
<br> - Second letter denotes a specific rurual region, an entire medium-sized city or a section of a major metropolitian area.
<br> - Third letter denotes an extract area if it is a city/town or other geographic area
<br> Local Delivery Unit (LDU) makes up the remaining characters of the postal code. This is for determining the specific building, city block for delivery. Therefore, the first 3 characters is sufficient in determining the approximate location.
<br>
<br>For a better depiction of the list of codes, please visit https://www.canadapost-postescanada.ca/cpc/en/support/articles/addressing-guidelines/postal-codes.page#:~:text=Two%20segments%20make%20up%20a,numeric%2Dalpha%2Dnumeric).

Source: https://en.wikipedia.org/wiki/Postal_codes_in_Canada

In [26]:
filtered_df_not_equal_7_3 = filtered_df_not_equal_7[filtered_df_not_equal_7['postal_code'].str.len() != 3]
print(len(filtered_df_not_equal_7_3))

108


There are 108 rows (or listings) that do not have consist 3 characters in their postal code.

In [102]:
filtered_df_not_equal_7_3[60:80]
# 1806 - 2008 Fullerton Avenue -> V7P

,mls_number,address,city,home_type,yr_built,home_age,garage,garage_size,taxes,avg_price_sqft,...,list_date,list_price,end_date,sold_price,difference_in_days,price_difference_abs,price_difference_pct,geocode_json,postal_code,index_col
10293,R2793090,341 N Dollarton Highway,vancouver,House,1974.0,50.0,No,4.0,"$6,240",$640,...,15/08/2023,"$1,799,900",09/09/2023,"$1,790,000",25.0,-9900.0,-0.550031,[{'address_components': [{'long_name': 'Dollar...,None,10293
10325,R2792117,978 Belmont Avenue,vancouver,House,1962.0,62.0,Yes,3.0,"$9,027","$1,386",...,04/09/2023,"$2,998,000",07/09/2023,"$3,000,000",3.0,2000.0,0.066711,[{'address_components': [{'long_name': 'Belmon...,None,10325
10326,R2811813,302 - 733 W 3rd Street,vancouver,Apartment/Condominium,2015.0,9.0,Yes,1.0,"$2,318","$1,024",...,04/09/2023,"$889,900",07/09/2023,"$889,000",3.0,-900.0,-0.101135,[{'address_components': [{'long_name': 'West 3...,None,10326
10336,R2784735,2366 Nelson Avenue,vancouver,House,1949.0,75.0,Yes,3.0,"$6,821","$1,146",...,04/09/2023,"$2,998,000",07/09/2023,"$2,850,000",3.0,-148000.0,-4.936624,[{'address_components': [{'long_name': 'Nelson...,None,10336
10356,R2810467,1015 - 2012 Fullerton Avenue,vancouver,Apartment/Condominium,1972.0,52.0,Yes,1.0,"$1,529",$687,...,28/08/2023,"$575,000",06/09/2023,"$598,900",9.0,23900.0,4.156522,"[{'address_components': [{'long_name': '1015',...",None,10356
10582,R2798338,220 - 2012 Fullerton Avenue,vancouver,Apartment/Condominium,1972.0,52.0,Yes,0.0,"$1,090",$641,...,13/07/2023,"$450,000",24/08/2023,"$450,000",42.0,0.0,0.000000,"[{'address_components': [{'long_name': '220', ...",None,10582
10789,R2806091,113 - 2020 Se Kent Avenue,vancouver,Apartment/Condominium,1994.0,30.0,Yes,1.0,"$2,014",$827,...,10/08/2023,"$725,000",15/08/2023,"$730,000",5.0,5000.0,0.689655,[{'address_components': [{'long_name': 'Vancou...,None,10789
10823,R2796849,105 - 1990 Se Kent Avenue,vancouver,Apartment/Condominium,1994.0,30.0,Yes,1.0,"$2,095",$771,...,09/07/2023,"$819,000",14/08/2023,"$812,000",36.0,-7000.0,-0.854701,[{'address_components': [{'long_name': 'Vancou...,None,10823
10943,R2805320,602 - 719 W 3rd Street,vancouver,Apartment/Condominium,2017.0,7.0,Yes,1.0,"$3,382","$1,061",...,09/08/2023,"$1,265,000",09/08/2023,"$1,265,000",0.0,0.0,0.000000,[{'address_components': [{'long_name': 'West 3...,None,10943
11109,R2798035,219 - 723 W 3rd Street,vancouver,Apartment/Condominium,2016.0,8.0,Yes,1.0,"$2,174","$1,000",...,25/07/2023,"$799,000",02/08/2023,"$789,000",8.0,-10000.0,-1.251564,[{'address_components': [{'long_name': 'West 3...,None,11109


Let's attempt to replace those "None" values to actual postal codes.

In [83]:
import geopy
geo_locator = geopy.Nominatim(user_agent='my_request')
                        # Latitude, Longitude
lat = filtered_df_not_equal_7_3['geocode_json'][14485][0]['geometry']['location']['lat']
lon = filtered_df_not_equal_7_3['geocode_json'][14485][0]['geometry']['location']['lng']
r = geo_locator.reverse((lat, lon))
print(r.raw['address']['postcode'])

V6Z 2H7


In [82]:
print(lat)
print(lon)

49.2071344
-123.057276


For some reason, this seems to be pointing to a specific building and multiple addresses are doing. This is not accurate. Since there are only 108 rows, let's try to manually fill the information.

In [98]:
enrich.regex_fix_misc(df_master)

In [93]:
df_master.loc[df_master['mls_number'] == 'R2818744','postal_code']

38    V5E 4M6
Name: postal_code, dtype: object

In [11]:
r

Location(Vancouver Art Gallery, 750, Hornby Street, Yaletown, Downtown, Vancouver, Metro Vancouver Regional District, British Columbia, V6Z 2H7, Canada, (49.28287695, -123.12056364787097, 0.0))

In [ ]:
# geocode_result = gmaps.geocode(df_master.address[0])

In [97]:
import importlib
importlib.reload(enrich)

<module 'src.data_enrichment' from 'C:\\Users\\Alan\\Projects\\Housing Project\\src\\data_enrichment.py'>